In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

from selenium import webdriver

In [62]:
def extract_product_offers(url):
    wrapper_class_name = 'styles__Wrapper-crf3j2-0 eFgqLx'
    product_wrapper_class = 'styles__ProductGridItem-crf3j2-1'

    wrapper_class_name = 'div.styles__Wrapper-crf3j2-0.eFgqLx'

    image_class_name = 'a.styles__CardMediaWrapper-sc-1gzprri-4.WLhYY'
    product_unavailable = 'span.styles__ProductIsUnavailable-sc-1idhk7x-9'
    discount_class = 'div.styles__WrapperProductCard-sc-1gzprri-10.irDXTp'
    driver = webdriver.Firefox() 
    driver.get(url)


    wrapper = driver.find_element_by_css_selector(wrapper_class_name)
    products = driver.find_elements_by_class_name(product_wrapper_class)

    products_df = pd.DataFrame([], columns=['Title', 'Url', 'Sdk', 'discount', 'unavailable'])
    for product in products:
        image = product.find_element_by_css_selector(image_class_name)
        unavailable = len(product.find_elements_by_css_selector(product_unavailable)) != 0

        discounts = image.find_elements_by_css_selector(discount_class)

        if (len(discounts) == 0):
            discount = None
        else:
            discount = discounts[0].get_attribute('innerText')
            discount = discount[1:] # Removes trailing "-"
        

        title = image.get_attribute('title')
        url = image.get_attribute('href')
        ind = url.rfind('/')+1
        sdk = url[ind:]
        products_df = products_df.append({'Title': title, 'Url': url, 'Sdk': sdk, 'discount': discount, 'unavailable': unavailable}, ignore_index=True)
    driver.quit()
    return products_df

In [60]:

refrigerators_url = 'https://www.extra.com.br/c/eletrodomesticos/refrigeradores/?filtro=c13_c14_c13&des=0TO25'

tvs_url = 'https://www.extra.com.br/c/tv-e-video/televisores/\
          ?filtro=c1_c2&nid=202485&des=0TO100'

printers_url = 'https://www.extra.com.br/c/informatica/z\
                impressoras/?filtro=c56_c61&des=0TO100'


In [61]:

refrigerators = extract_product_offers(refrigerators_url)
tvs = extract_product_offers(tvs_url)
printers = extract_product_offers(printers_url)


NoSuchElementException: Message: Unable to locate element: div.styles__WrapperProductCard-sc-1gzprri-10.irDXTp


In [56]:
refrigerators

,Title,Url,Sdk,discount,unavailable
0,Refrigerador Consul Frost Free CRM43NK com 2 P...,https://www.extra.com.br/refrigerador-consul-f...,9600890,-4%,False
1,Refrigerador Consul Frost Free Facilite CRB39A...,https://www.extra.com.br/refrigerador-consul-f...,10153519,-7%,False
2,Refrigerador Consul Frost Free CRM39AB Duplex ...,https://www.extra.com.br/refrigerador-consul-f...,50004596,-4%,False
3,Refrigerador Electrolux DF44 com Prateleira Re...,https://www.extra.com.br/refrigerador-electrol...,15243319,-7%,False
4,Refrigerador Electrolux Frost Free DB53 Bottom...,https://www.extra.com.br/refrigerador-electrol...,11688788,-3%,False
5,Refrigerador Electrolux DF56 com Icemax Branco...,https://www.extra.com.br/refrigerador-electrol...,15243314,-6%,False
6,Refrigerador Brastemp Side Inverse BRO80AB com...,https://www.extra.com.br/refrigerador-brastemp...,4420011,-4%,False
7,Refrigerador Panasonic Frost Free NR-BB53GV3W ...,https://www.extra.com.br/refrigerador-panasoni...,55007189,-4%,False
8,Refrigerador Consul CRE44AB Frost Free Duplex ...,https://www.extra.com.br/refrigerador-consul-c...,50003891,-3%,True
9,Refrigerador Electrolux Infinity DF80X Frost F...,https://www.extra.com.br/refrigerador-electrol...,9344,-13%,True


In [37]:
tvs

,Title,Url,Sdk
0,Refrigerador Consul Frost Free CRM43NK com 2 P...,https://www.extra.com.br/refrigerador-consul-f...,9600890
1,Refrigerador Consul Frost Free Facilite CRB39A...,https://www.extra.com.br/refrigerador-consul-f...,10153519
2,Refrigerador Consul Frost Free CRM39AB Duplex ...,https://www.extra.com.br/refrigerador-consul-f...,50004596
3,Refrigerador Electrolux DF44 com Prateleira Re...,https://www.extra.com.br/refrigerador-electrol...,15243319
4,Refrigerador Electrolux Frost Free DB53 Bottom...,https://www.extra.com.br/refrigerador-electrol...,11688788
5,Refrigerador Electrolux DF56 com Icemax Branco...,https://www.extra.com.br/refrigerador-electrol...,15243314
6,Refrigerador Brastemp Side Inverse BRO80AB com...,https://www.extra.com.br/refrigerador-brastemp...,4420011
7,Refrigerador Panasonic Frost Free NR-BB53GV3W ...,https://www.extra.com.br/refrigerador-panasoni...,55007189
8,Refrigerador Consul CRE44AB Frost Free Duplex ...,https://www.extra.com.br/refrigerador-consul-c...,50003891
9,Refrigerador Electrolux Infinity DF80X Frost F...,https://www.extra.com.br/refrigerador-electrol...,9344
